# What’s a Customer Worth? Modelling Customers Lifetime Value

In [1]:
import warnings
warnings.simplefilter('ignore')
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import pgeocode
nomi = pgeocode.Nominatim('IN')
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split as tts,cross_val_score,ShuffleSplit
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score,f1_score,classification_report,roc_auc_score
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)

In [2]:
data = pd.read_csv(r'C:\Users\ckatkaa\Desktop\jupyter notebook\Capstone project  Mahindra Group\Clean Data\Clean Data set\clean.csv')

In [3]:
data.head()

,Unnamed: 0,Unnamed: 0_x,Unnamed: 0.1,Cust Type,Customer No.,Gate Pass Time,Invoice Date,Invoice No,Invoice Time,Job Card No,JobCard Date,JobCard Time,KMs Reading,Labour Total,Make,Misc Total,Model,OSL Total,Order Type,Parts Total,Pin code,Plant,Plant Name1,Print Status,Recovrbl Exp,Total Amt Wtd Tax.,User ID,Clean_CITY,Clean_District,Unnamed: 0_y,Business Partner,Partner Type,Data Origin,Unnamed: 0.1.1,DBM Order,Item Category,Net value,Order Quantity
0,0,0,0,Retail,67849,00:00:00,2015-01-02,7005200002,11:30:36,168303,2014-12-13,14:29:43,49317,1203.14,GENERAL MOTORS,0.00,SPARK,500.06,Paid Service,2348.75,400601,BC01,THANE,NO,0.0,4051.95,BC01FS1,Thane,Maharashtra,NaN,NaN,NaN,NaN,339512.0,168303.0,P001,1203.14,73.0
1,1,0,0,Retail,67849,00:00:00,2015-01-02,7005200002,11:30:36,168303,2014-12-13,14:29:43,49317,1203.14,GENERAL MOTORS,0.00,SPARK,500.06,Paid Service,2348.75,400601,BC01,THANE,NO,0.0,4051.95,BC01FS1,Thane,Maharashtra,NaN,NaN,NaN,NaN,339513.0,168303.0,P002,2348.75,6.5
2,2,0,0,Retail,67849,00:00:00,2015-01-02,7005200002,11:30:36,168303,2014-12-13,14:29:43,49317,1203.14,GENERAL MOTORS,0.00,SPARK,500.06,Paid Service,2348.75,400601,BC01,THANE,NO,0.0,4051.95,BC01FS1,Thane,Maharashtra,NaN,NaN,NaN,NaN,339514.0,168303.0,P010,500.06,1.0
3,3,1,1,Retail,84419,00:00:00,2015-01-03,7005200003,10:07:32,173997,2015-01-02,14:12:18,78584,804.26,TATA MOTORS,197.03,INDICA,0.00,SMC Value Package,0.00,400603,BC01,THANE,NO,0.0,1001.29,BC01SA2,Thane,Maharashtra,NaN,NaN,NaN,NaN,351037.0,173997.0,P001,804.26,170.0
4,4,1,1,Retail,84419,00:00:00,2015-01-03,7005200003,10:07:32,173997,2015-01-02,14:12:18,78584,804.26,TATA MOTORS,197.03,INDICA,0.00,SMC Value Package,0.00,400603,BC01,THANE,NO,0.0,1001.29,BC01SA2,Thane,Maharashtra,NaN,NaN,NaN,NaN,351038.0,173997.0,P011,197.03,1.0


In [4]:
data =  data.drop(columns=['Unnamed: 0','Unnamed: 0_x','Unnamed: 0.1'],axis = 1)

In [5]:
data["Invoice_DateTime"]=data["Invoice Date"]+' '+data["Invoice Time"]
data["JobCard_DateTime"]=data["JobCard Date"]+' '+data["JobCard Time"]
data['JobCard_DateTime'] = pd.to_datetime(data['JobCard_DateTime'],dayfirst=True)
data['Invoice_DateTime'] = pd.to_datetime(data['Invoice_DateTime'],dayfirst=True)
data["Service_Time"]=data["Invoice_DateTime"]-data["JobCard_DateTime"]

In [6]:
data['year'] = pd.to_datetime(data['Invoice_DateTime']).dt.year
data['months'] = pd.to_datetime(data['Invoice_DateTime']).dt.month

In [7]:
data['Service_Time'].head()

0   19 days 21:00:53
1   19 days 21:00:53
2   19 days 21:00:53
3    0 days 19:55:14
4    0 days 19:55:14
Name: Service_Time, dtype: timedelta64[ns]

In [8]:
data["Invoice_DateTime"]=pd.to_datetime(data["Invoice_DateTime"],dayfirst=True)
data["JobCard_DateTime"]=pd.to_datetime(data["JobCard_DateTime"],dayfirst=True)
data['Service_Time'] = data['Invoice_DateTime'] - data['JobCard_DateTime']

In [9]:
# Calculating service hrs
data['Service_Time_Hrs'] = data['Service_Time']/np.timedelta64(1,'h')

In [10]:
data['Service_Time_Hrs'].describe()

count    936275.000000
mean         89.878782
std         275.095915
min          -0.009167
25%           4.048611
50%          23.258889
75%          73.230278
max       17769.261389
Name: Service_Time_Hrs, dtype: float64

In [11]:
data[data['Service_Time_Hrs']<0]

,Cust Type,Customer No.,Gate Pass Time,Invoice Date,Invoice No,Invoice Time,Job Card No,JobCard Date,JobCard Time,KMs Reading,Labour Total,Make,Misc Total,Model,OSL Total,Order Type,Parts Total,Pin code,Plant,Plant Name1,Print Status,Recovrbl Exp,Total Amt Wtd Tax.,User ID,Clean_CITY,Clean_District,Unnamed: 0_y,Business Partner,Partner Type,Data Origin,Unnamed: 0.1.1,DBM Order,Item Category,Net value,Order Quantity,Invoice_DateTime,JobCard_DateTime,Service_Time,year,months,Service_Time_Hrs
714423,Retail,182085,00:00:00,2016-05-02,7167000988,15:12:11,363294,2016-05-02,15:12:44,62001,0.0,MARUTI SUZUKI,0.0,SWIFT DZIRE,0.0,Running Repairs,0.0,603103,X085,X085 - SRI MOTORS,NO,0.0,0.0,X085SA1,Kanchipuram,Tamil Nadu,NaN,NaN,NaN,NaN,711636.0,363294.0,G2TX,0.0,0.0,2016-05-02 15:12:11,2016-05-02 15:12:44,-1 days +23:59:27,2016,5,-0.009167


In [12]:
data.drop(index=714423,axis=0,inplace=True)

In [13]:
data['year'].value_counts()

2016    337170
2015    248353
2014    201456
2013    112627
2012     36668
Name: year, dtype: int64

# Customers Lifetime Value for each customer over a period of 2016 (1 year) based on "Make" and "Model" of the cars

In [14]:
data_16=data[data["year"]==2016]

In [15]:
data_16.shape

(337170, 41)

In [16]:
data_16.head()

,Cust Type,Customer No.,Gate Pass Time,Invoice Date,Invoice No,Invoice Time,Job Card No,JobCard Date,JobCard Time,KMs Reading,Labour Total,Make,Misc Total,Model,OSL Total,Order Type,Parts Total,Pin code,Plant,Plant Name1,Print Status,Recovrbl Exp,Total Amt Wtd Tax.,User ID,Clean_CITY,Clean_District,Unnamed: 0_y,Business Partner,Partner Type,Data Origin,Unnamed: 0.1.1,DBM Order,Item Category,Net value,Order Quantity,Invoice_DateTime,JobCard_DateTime,Service_Time,year,months,Service_Time_Hrs
599104,Retail,119371,15:41:33,2016-01-02,7005202789,11:27:27,300412,2016-01-02,10:52:24,35654,1125.76,MARUTI SUZUKI,291.45,ZEN ESTILO,0.0,SMC Value Package,2082.07,400089,BC01,THANE,YES,0.0,3499.28,BC01SA2,Mumbai,Maharashtra,NaN,NaN,NaN,NaN,600047.0,300412.0,P001,1125.76,2.0,2016-01-02 11:27:27,2016-01-02 10:52:24,00:35:03,2016,1,0.584167
599105,Retail,119371,15:41:33,2016-01-02,7005202789,11:27:27,300412,2016-01-02,10:52:24,35654,1125.76,MARUTI SUZUKI,291.45,ZEN ESTILO,0.0,SMC Value Package,2082.07,400089,BC01,THANE,YES,0.0,3499.28,BC01SA2,Mumbai,Maharashtra,NaN,NaN,NaN,NaN,600048.0,300412.0,P002,2082.07,10.0,2016-01-02 11:27:27,2016-01-02 10:52:24,00:35:03,2016,1,0.584167
599106,Retail,119371,15:41:33,2016-01-02,7005202789,11:27:27,300412,2016-01-02,10:52:24,35654,1125.76,MARUTI SUZUKI,291.45,ZEN ESTILO,0.0,SMC Value Package,2082.07,400089,BC01,THANE,YES,0.0,3499.28,BC01SA2,Mumbai,Maharashtra,NaN,NaN,NaN,NaN,600049.0,300412.0,P011,291.45,1.0,2016-01-02 11:27:27,2016-01-02 10:52:24,00:35:03,2016,1,0.584167
599107,Retail,128929,13:05:33,2016-01-02,7005202790,13:03:16,300447,2016-01-02,11:50:58,70730,233.85,MARUTI SUZUKI,0.00,SWIFT,0.0,Running Repairs,195.44,400604,BC01,THANE,YES,0.0,429.29,BC01SA2,Thane,Maharashtra,NaN,NaN,NaN,NaN,600118.0,300447.0,P001,233.85,36.0,2016-01-02 13:03:16,2016-01-02 11:50:58,01:12:18,2016,1,1.205000
599108,Retail,128929,13:05:33,2016-01-02,7005202790,13:03:16,300447,2016-01-02,11:50:58,70730,233.85,MARUTI SUZUKI,0.00,SWIFT,0.0,Running Repairs,195.44,400604,BC01,THANE,YES,0.0,429.29,BC01SA2,Thane,Maharashtra,NaN,NaN,NaN,NaN,600119.0,300447.0,P002,195.44,1.0,2016-01-02 13:03:16,2016-01-02 11:50:58,01:12:18,2016,1,1.205000


In [17]:
# Calculating frequency of visits & total revenue of a customer for calculation of ltv

customer_freq=data_16.groupby(["Make","Model","Clean_CITY","Customer No."]).agg({"Invoice No":'nunique',"Total Amt Wtd Tax.":'mean',"Service_Time_Hrs":'mean'})
customer_freq=customer_freq.reset_index()

customer_freq.head()

clv_model=customer_freq.copy()
clv_model["CLV"] = clv_model["Invoice No"] * clv_model["Total Amt Wtd Tax."]
clv_model.head()

,Make,Model,Clean_CITY,Customer No.,Invoice No,Total Amt Wtd Tax.,Service_Time_Hrs,CLV
0,AUDI,A4,Ahmednagar,129425,1,1921.40,361.850833,1921.40
1,AUDI,A4,Bengaluru,197826,1,0.00,839.515278,0.00
2,AUDI,A4,Chennai,251998,1,131637.61,359.626667,131637.61
3,AUDI,A4,Dharmapuri,99532,5,4660.08,60.879889,23300.40
4,AUDI,A4,East Godavari,182988,2,479.38,12.871528,958.76


Selecting the customers for analysis

In [18]:
clv_model = clv_model[clv_model['CLV'] > 0]

In [19]:
print('Count of customers are under our analysis :', clv_model.shape)

Count of customers are under our analysis : (124613, 8)


# Analysis of CLV 

In [20]:
# Quantile based analysis of LTV to decide threshold
print("0.05 quantile :-",clv_model['CLV'].quantile(0.05))
print("0.25 quantile :-",clv_model['CLV'].quantile(0.25))
print("0.50 quantile :-",clv_model['CLV'].quantile(0.50))
print("0.75 quantile :-",clv_model['CLV'].quantile(0.75))
print("0.95 quantile :-",clv_model['CLV'].quantile(0.95))
print("minimum LTV :-",clv_model['CLV'].min())
print("maximum LTV :-",clv_model['CLV'].max())
print("mean LTV :-",clv_model['CLV'].mean())

0.05 quantile :- 99.14
0.25 quantile :- 465.67
0.50 quantile :- 2334.12
0.75 quantile :- 6750.25
0.95 quantile :- 29514.164666666606
minimum LTV :- 0.01
maximum LTV :- 1790743.6553142855
mean LTV :- 7369.717702992323


In [21]:
# Sepreting into Low,Medium and High on the basis of quantile values
bins=[-1,2334.12,6750.25,1790743.6553142855]
labels=["Low","Medium","High"]
clv_model["CLV Binned"]=pd.cut(clv_model["CLV"], bins=bins, labels=labels)

In [22]:
clv_model.head()

,Make,Model,Clean_CITY,Customer No.,Invoice No,Total Amt Wtd Tax.,Service_Time_Hrs,CLV,CLV Binned
0,AUDI,A4,Ahmednagar,129425,1,1921.40,361.850833,1921.40,Low
2,AUDI,A4,Chennai,251998,1,131637.61,359.626667,131637.61,High
3,AUDI,A4,Dharmapuri,99532,5,4660.08,60.879889,23300.40,High
4,AUDI,A4,East Godavari,182988,2,479.38,12.871528,958.76,Low
5,AUDI,A4,East Godavari,243916,1,1478.30,4.828611,1478.30,Low


In [23]:
clv_model["CLV Binned"].value_counts()

Low       62307
High      31153
Medium    31153
Name: CLV Binned, dtype: int64

In [24]:
clv_make_model=clv_model.copy()

In [25]:
le=LabelEncoder()
clv_make_model["Make"]=le.fit_transform(clv_make_model["Make"])
clv_make_model["Model"]=le.fit_transform(clv_make_model["Model"])
clv_make_model["CLV Binned"]=le.fit_transform(clv_make_model["CLV Binned"])
clv_make_model["Clean_CITY"]=le.fit_transform(clv_make_model["Clean_CITY"])

In [26]:
clv_make_model.head()

,Make,Model,Clean_CITY,Customer No.,Invoice No,Total Amt Wtd Tax.,Service_Time_Hrs,CLV,CLV Binned
0,0,8,18,129425,1,1921.40,361.850833,1921.40,1
2,0,8,145,251998,1,131637.61,359.626667,131637.61,0
3,0,8,188,99532,5,4660.08,60.879889,23300.40,0
4,0,8,200,182988,2,479.38,12.871528,958.76,1
5,0,8,200,243916,1,1478.30,4.828611,1478.30,1


In [27]:
# spliiting the data set
X=clv_make_model.drop(["Customer No.","CLV","CLV Binned","Invoice No"],1)
y=clv_make_model["CLV Binned"]

In [28]:
X_train,X_test,y_train,y_test=tts(X,y,test_size=0.3,random_state=0)

Random Forest Classifier

In [29]:
rf=RandomForestClassifier(criterion='entropy',max_depth=8,random_state=0)
rf.fit(X_train,y_train)
y_pred_rf=rf.predict(X_test)
y_pred_rf_train = rf.predict(X_train)
print(classification_report(y_test,y_pred_rf))


              precision    recall  f1-score   support

           0       0.99      0.71      0.83      9288
           1       0.90      1.00      0.95     18699
           2       0.75      0.80      0.78      9397

    accuracy                           0.88     37384
   macro avg       0.88      0.84      0.85     37384
weighted avg       0.89      0.88      0.88     37384



In [30]:
train_acc = rf.score(X_train,y_train)
print("Training score: %.2f " % train_acc)
test_acc = rf.score(X_test,y_test)
print("Testing score: %.2f " % test_acc)     

Training score: 0.88 
Testing score: 0.88 


In [31]:
print("Training Accuracy :-" , accuracy_score(y_test,y_pred_rf))
print("Testing Accuracy :-" ,accuracy_score(y_train,y_pred_rf_train))

Training Accuracy :- 0.8784239246736572
Testing Accuracy :- 0.8776897591397357


In [32]:
#Cross Validation Score

rf_clf_cv_score=cross_val_score(rf,X,y,cv=5)

print(rf_clf_cv_score)
print(rf_clf_cv_score.mean())

[0.87802431 0.86133291 0.88203667 0.86168847 0.74388091]
0.8453926539291132


XgBoost Classifier

In [33]:
xgb=XGBClassifier()
xgb.fit(X_train,y_train)
y_pred_xgb=xgb.predict(X_test)

print(classification_report(y_test,y_pred_xgb))

              precision    recall  f1-score   support

           0       0.95      0.79      0.86      9288
           1       0.92      0.99      0.95     18699
           2       0.79      0.81      0.80      9397

    accuracy                           0.89     37384
   macro avg       0.89      0.86      0.87     37384
weighted avg       0.90      0.89      0.89     37384



In [34]:
train_acc_xgb = xgb.score(X_train,y_train)
print("Training score: %.2f " % train_acc)
test_acc_xgb = xgb.score(X_test,y_test)
print("Testing score: %.2f " % test_acc)

Training score: 0.88 
Testing score: 0.88 


In [35]:
#Cross Validation Score

xgb_cv_score=cross_val_score(xgb,X,y,cv=5)
print(xgb_cv_score.mean())

0.8252666559041492


One-Vs-Rest for Multi-Class Classification

In [36]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer

xgb1 = XGBClassifier(num_classes=3)
ovr = OneVsRestClassifier(xgb1)

# Fit the model on train data
ovr.fit(X_train,y_train)

# predict on test data
ovr_y_pred = ovr.predict(X_test)
print(classification_report(y_test,ovr_y_pred))

[09:41:57] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { num_classes } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[09:42:15] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { num_classes } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[09:42:33] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { num_classes } might not be used.

  This may not be accurate due to some parameters are only used in langu

In [37]:
ovr_y_pred

array([1, 1, 1, ..., 2, 2, 1])

SVM classifier

In [ ]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train)
y_pred_svc = svclassifier.predict(X_test)
print(classification_report(y_test,y_pred_svc))

In [ ]:
#Cross Validation Score

svclassifier_cv_score=cross_val_score(svclassifier,X,y,cv=5)
print(svclassifier_cv_score.mean())